In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from UnarySim.sw.kernel.mul import UnaryMul
from UnarySim.sw.stream.gen import RNG, SourceGen, BSGen
from UnarySim.sw.metric.metric import ProgressiveError
import matplotlib.pyplot as plt
import time
import math

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
rng = "Sobol"

In [4]:
col = 8
modes = ["bipolar"]
static = True
bitwidth = 8
stype = torch.int8
rtype = torch.long

for mode in modes:
    if mode == "unipolar":
        input_prob = torch.nn.Parameter(torch.rand(col).mul(2**bitwidth).round().div(2**bitwidth)).to(device)
        iVec = torch.rand(col).mul(2**bitwidth).round().div(2**bitwidth).to(device)
    elif mode == "bipolar":
        input_prob =  torch.nn.Parameter(torch.rand(col).mul(2).sub(1).mul(2**bitwidth).round().div(2**bitwidth)).to(device)
        iVec = torch.rand(col).mul(2).sub(1).mul(2**bitwidth).round().div(2**bitwidth).to(device)

    dut_mul = UnaryMul(bitwidth=bitwidth, mode=mode, static=static, input_prob_1=input_prob, stype=stype, rtype=rtype).to(device)

    oVec = torch.mul(iVec, input_prob).mul(2**bitwidth).round().div(2**bitwidth).to(device)

    iVecPE = ProgressiveError(iVec, mode=mode).to(device)
    oVecPE = ProgressiveError(oVec, mode=mode).to(device)

    iVecSource = SourceGen(iVec, bitwidth, mode=mode)().to(device)
    iVecRNG = RNG(bitwidth, 1, rng)().to(device)
    iVecBS = BSGen(iVecSource, iVecRNG, stype).to(device)

    with torch.no_grad():
        start_time = time.time()
        for i in range(2**bitwidth):             # unary cycle count 2^n for n-bit binary data
            iBS = iVecBS(torch.tensor([i]))      # input bit stream generation
            iVecPE.Monitor(iBS)                  # input accuracy measurement
            oVecU = dut_mul(iBS)                 # computing kernel, e.g., multiplication
            oVecPE.Monitor(oVecU)                # output accuracy measurement
    
    print(oVecPE())
    
    bin_sum = torch.sum(oVec)
    unary_sum = torch.sum(oVecPE()[1])
    print(bin_sum, unary_sum)

(Parameter containing:
tensor([ 0.0000,  0.1875, -0.0312, -0.2031, -0.2500,  0.0469,  0.0312, -0.2500],
       device='cuda:0'), Parameter containing:
tensor([ 0.0039,  0.0078, -0.0039,  0.0000,  0.0039, -0.0234,  0.0000, -0.0078],
       device='cuda:0'))
tensor(-0.4492, device='cuda:0', grad_fn=<SumBackward0>) tensor(-0.0195, device='cuda:0')
